# The Simpsons Characters

## Requirements

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
from torchvision.datasets import ImageFolder
import torchinfo

## Constants

In [ ]:
lr = 0.001
batch_size = 128
class_number = 42
epochs = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## NN model

In [4]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32,
                      kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64,
                      kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())

        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64,
                      kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=128,
                      kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())

        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=128,
                      kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=256,
                      kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())

        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=256,
                      kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=512,
                      kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())

        self.conv5 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=512,
                      kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512,
                      kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(512, 42)

    def forward(self, input):
        output = self.conv1(input)
        output = self.conv2(output)
        output = self.conv3(output)
        output = self.conv4(output)
        output = self.conv5(output)
        output = self.pool(output)
        output = output.view(-1, 512)
        output = self.fc1(output)
        return output

    def saveModel(self, filename='./simpsons_model.pth'):
        torch.save(obj=self.state_dict(), f=filename)

    def train(self, optimizer, train_loader, criterion, epochs):
        print('Training started')

        for epoch in range(epochs):
            running_loss = 0.0
            for step, data in enumerate(train_loader, 0):
                inputs, labels = data[0].to(device), data[1].to(device)

                optimizer.zero_grad()
                outputs = self(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                if step % 10 == 9:    # print every 10 mini-batches
                    print(
                        f'[{epoch + 1}, {step + 1:5d}] loss: {running_loss/10:.3f}')
                    running_loss = 0.0

        print('Training finished')

    def valid(self):
        print('Validation started')
        print('Validation finished')
        pass

    def test(self, test_loader):
        print('Testing started')
        dataiter = iter(test_loader)
        images, labels = next(dataiter)

        correct = 0
        total = 0
        with torch.no_grad():
            for data in test_loader:
                images, labels = data[0].to(device), data[1].to(device)
                
                outputs = self(images)
    
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'Accuracy: {100 * correct // total}%')
        print('Testing finished')


## Optimizer and Loss function

In [ ]:
model = Network()
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
criterion = nn.CrossEntropyLoss()

## Data dividing

In [3]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                torchvision.transforms.Normalize(mean=[0.5], std=[0.5]),
                                                torchvision.transforms.Resize((32,32))])
dataset = ImageFolder(
    "The Simpsons Characters Data\\kaggle_simpson_testset", transform=transform)

train_set, test_set = random_split(dataset, [0.7, 0.3])
#valid_set = ImageFolder ("The Simpsons Characters Data\\valid",transform=transform)

train_loader=torch.utils.data.DataLoader(train_set,batch_size=batch_size,shuffle=True, num_workers=1)
test_loader = torch.utils.data.DataLoader(test_set,batch_size=batch_size,shuffle=True, num_workers=1)
#valid_loader = torch.utils.data.DataLoader(valid_set,batch_size=batch_size,shuffle=True, num_workers=1)

## Training

In [ ]:
model.train(optimizer,train_loader,criterion,epochs)

## Validation

## Test

In [ ]:
model.test(test_loader)